In [62]:
import torch
import torch.nn as nn
from load_data import *
import numpy as np
from torch.optim import Adam
from tqdm import tqdm


In [63]:
class Encoder(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(Encoder, self).__init__()

        self.FC_input = nn.Linear(input_dim, hidden_dim)
        self.FC_input2 = nn.Linear(hidden_dim, hidden_dim)
        self.FC_latent  = nn.Linear(hidden_dim, latent_dim)

        
        self.LeakyReLU = nn.LeakyReLU(0.2)
        
        self.training = True
        
    def forward(self, x):
        h_       = self.LeakyReLU(self.FC_input(x))
        h_       = self.LeakyReLU(self.FC_input2(h_))
        latent     = self.FC_latent(h_)

        
        return latent

    
class Decoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, output_dim):
        super(Decoder, self).__init__()
        self.FC_hidden = nn.Linear(latent_dim, hidden_dim)
        self.FC_hidden2 = nn.Linear(hidden_dim, hidden_dim)
        self.FC_output = nn.Linear(hidden_dim, output_dim)
        
        self.LeakyReLU = nn.LeakyReLU(0.2)
        
    def forward(self, x):
        h     = self.LeakyReLU(self.FC_hidden(x))
        h     = self.LeakyReLU(self.FC_hidden2(h))
        
        x_hat = self.FC_output(h)
        return x_hat
    
class Model(nn.Module):
    def __init__(self, Encoder, Decoder):
        super(Model, self).__init__()
        self.Encoder = Encoder
        self.Decoder = Decoder
        
                
    def forward(self, x):
        latent = self.Encoder(x)
        x_hat = self.Decoder(latent)
        
        return x_hat, latent

In [64]:
x_dim = 20
hidden_dim = 18
latent_dim = 15
encoder = Encoder(input_dim=x_dim, hidden_dim=hidden_dim, latent_dim=latent_dim)
decoder = Decoder(latent_dim=latent_dim, hidden_dim = hidden_dim, output_dim =x_dim)

movie_model = Model(Encoder=encoder, Decoder=decoder)


x_dim = 3
hidden_dim = 10
latent_dim = 15
encoder = Encoder(input_dim=x_dim, hidden_dim=hidden_dim, latent_dim=latent_dim)
decoder = Decoder(latent_dim=latent_dim, hidden_dim = hidden_dim, output_dim =x_dim)

user_model = Model(Encoder=encoder, Decoder=decoder)

In [65]:
movie_features = load__movies_info()
user_movie_ratings = load_user_movie_rating()
user_features = load_users_info()

avg_score = user_movie_ratings.groupby("movie id").mean()['rating'].sort_values(ascending=False)
movie_features=pd.merge(movie_features,avg_score,on="movie id")
movie_features = movie_features.drop(['IMDb URL', 'video release date', 'release date', 'movie title', 'movie id'], axis=1)
movies = movie_features.to_numpy()

user_features["gender"] = user_features["gender"].astype('category')
user_features["gender"] = user_features["gender"].cat.codes
user_features["occupation"] = user_features["occupation"].astype('category')
user_features["occupation"] = user_features["occupation"].cat.codes
user_features = user_features.drop(["zip code", 'user id'], axis=1)
users = user_features.to_numpy().astype(float)

In [66]:
loss_f = nn.MSELoss()
optimizer = Adam(movie_model.parameters(), lr=1e-3)


print("Start training VAE...")
movie_model.train()

for epoch in range(5):
    overall_loss = 0
    
    for i in range(movies.shape[0]):

        x = torch.from_numpy(movies[i].reshape(1, -1)).float()
        optimizer.zero_grad()

        x_recon, _ = movie_model(x)
        loss = loss_f(x_recon, x)
        
#         overall_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        
    print("\tEpoch", epoch + 1, "complete!", "\tAverage Loss: ", overall_loss / (movies.shape[0]))

print("Finish!!")

torch.save(movie_model.state_dict(), 'movie_vae.pt')

Start training VAE...
	Epoch 1 complete! 	Average Loss:  0.0
	Epoch 2 complete! 	Average Loss:  0.0
	Epoch 3 complete! 	Average Loss:  0.0
	Epoch 4 complete! 	Average Loss:  0.0
	Epoch 5 complete! 	Average Loss:  0.0
Finish!!


In [67]:
loss_f = nn.MSELoss()
optimizer = Adam(user_model.parameters(), lr=1e-3)


print("Start training VAE...")
movie_model.train()

for epoch in range(5):
    overall_loss = 0
    
    for i in range(users.shape[0]):

        x = torch.from_numpy(users[i].reshape(1, -1)).float()
        optimizer.zero_grad()

        x_recon, _ = user_model(x)
        loss = loss_f(x_recon, x)
        
#         overall_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        
    print("\tEpoch", epoch + 1, "complete!", "\tAverage Loss: ", overall_loss / (users.shape[0]))

print("Finish!!")

torch.save(user_model.state_dict(), 'user_vae.pt')

Start training VAE...
	Epoch 1 complete! 	Average Loss:  0.0
	Epoch 2 complete! 	Average Loss:  0.0
	Epoch 3 complete! 	Average Loss:  0.0
	Epoch 4 complete! 	Average Loss:  0.0
	Epoch 5 complete! 	Average Loss:  0.0
Finish!!


In [68]:
movies.shape

(1682, 20)